In [8]:
import astropy.table as atpy
import astropy.io.fits as pyfits
import astropy.wcs as pywcs
import numpy as np
import scipy.interpolate
import scipy.stats
betw = lambda x,x1,x2:(x>x1)&(x<=x2)

In [9]:
prefix='/tmp/' # prefix to the data files

#read the data file and compute the grid points
dat, hdr = pyfits.getdata(prefix+'n_nsc1.fits', header=True)
nw, nx1, nx2, nx3 = dat.T.shape
wc = pywcs.WCS(hdr)
x1, x2, x3 = np.mgrid[0:nx1, 0:nx2, 0:nx3]
_, y1, y2, y3 = wc.all_pix2world(np.array([x1.flatten() * 0, x1.flatten(),
                                           x2.flatten(), x3.flatten()]).T, 0).T


In [10]:
#build first interpolator
dat0 = dat.T.reshape(nw, -1)
II = scipy.interpolate.LinearNDInterpolator(np.array([y3, y2, y1]).T, dat0.T)
tab1 = atpy.Table().read(prefix+'run1.ipf', format='ascii')
Ivals = II(tab1['col2'], tab1['col3'], tab1['col4'])

In [11]:
#build second interpolator

tab2 = atpy.Table().read(prefix+'run2.ipf', format='ascii')
II2 = scipy.interpolate.LinearNDInterpolator(np.array([tab1['col2'],
                                                       tab1['col3'],
                                                       tab1['col4']]).T, Ivals)

In [18]:
# compute the values of the second interpolator 
# accumulate the array of original noninterp values
Ivals2 = II2(tab2['col2'], tab2['col3'], tab2['col4'])
res=[]
for i in range(len(tab2['col2'])):
        xind = (y3==tab2['col2'][i]) & (y2==tab2['col3'][i]) & (y1==tab2['col4'][i])
        assert(xind.sum() == 1)
        res.append(dat0[:,xind].flatten())
res = np.array(res)
                                                                

In [19]:
waves = 10**(hdr['CRVAL1'] + np.arange(nw) * hdr['CDELT1'])
sub = betw(waves, 4000, 10000)
def robust_std(x):
        return 0.5 * (scipy.stats.scoreatpercentile(x, 100-  16)-scipy.stats.scoreatpercentile(x,16))

print ('med', np.median((Ivals2-res)[:,sub]))
print ('std', np.std((Ivals2-res)))
print ('robstd', robust_std((Ivals2-res)))

med -0.0186406075954
std 0.0846237072588
robstd 0.02941860497
